# Abstract words

## Generate original fields

These are downloaded from sources.

In [1]:
# Function used:
from abslithist.words.fields import gen_fields_and_norms
gen_fields_and_norms??

Signature: gen_fields_and_norms()
Docstring: <no docstring>
Source:   
def gen_fields_and_norms():
    # init
    fields=defaultdict(set)
    norms=[]

    # add fields (so far, only quant/scale-based ones)
    funcs=[
        add_fields_paivio, # Paivo et al
        add_fields_mrc,    # MRC
        add_fields_brys,   # Brysbaert et al
        add_fields_lsn     # LSN
    ]

    # run through functions
    for func in tqdm(funcs,desc='Building fields and norms from sources'):
        func(fields,norms)

    # save fields
    fields_save=dict((k,list(v)) for k,v in fields.items())
    with open(PATH_FIELDS_JSON,'w') as of: json.dump(fields_save, of)
    
    # save norms
    qdf=pd.DataFrame(norms)
    qdf=qdf.drop_duplicates(['word','source'],keep='first')
    qdf.to_csv(PATH_NORMS,index=False)
File:      ~/github/abslithist/abslithist/words/fields.py
Type:      function


In [2]:
# gen_fields_and_norms()

## Filter them through semantic models

### First compute them

That is, find the word-to-field cosine distances. Just store them in the data/models directory as csv's (```word2field_dists.csv```).

In [3]:
from abslithist.models.embeddings import compute_vecfields
compute_vecfields??

Signature: compute_vecfields()
Docstring: <no docstring>
Source:   
def compute_vecfields():
    #from abslithist.models.embeddings import get_model_paths,compute_word2field_dists
    from abslithist.words.fields import get_origfields

    # get paths
    pathld = get_model_paths()
    
    # load fields
    fields = get_origfields()

    # compute
    objs = [(pathd,fields) for pathd in pathld]
    all_data = pmap(compute_word2field_dists, objs, num_proc=4, desc='Computing word to field distances')
File:      ~/github/abslithist/abslithist/models/embeddings.py
Type:      function


In [4]:
# compute_vecfields()

### Generate vector-based norms

Using the word2field distances generate above, loop over each distance table (corresponding to a model run), standardize all distances (zscore) within models, and then save the standardized distances in a manner similar to how the original word norms were saved.

In [5]:
from abslithist.words.fields import gen_vecnorms
gen_vecnorms??

Signature: gen_vecnorms()
Docstring: <no docstring>
Source:   
def gen_vecnorms():
    # paths
    from abslithist.models.embeddings import get_model_paths
    paths_ld = get_model_paths(model_fn='word2field_dists.csv')
    paths_df = pd.DataFrame(paths_ld)

    # get fields
    fields = get_origfields()

    # group runs
    groups=[
        group_df.to_dict('records')
        for gi,group_df in paths_df.groupby(['corpus','period_start','period_end'])
    ]

    pmap(_calc_norms_dist_group,groups,desc='Generating norms across model-periods')
File:      ~/github/abslithist/abslithist/words/fields.py
Type:      function


In [ ]:
gen_vecnorms()

Generating norms across model-periods:   0%|          | 0/68 [00:00<?, ?it/s]

### Break across fields